In [0]:
%pip install pandas
%pip install openpyxl
%pip install rich

In [0]:
import pandas as pd

# Pandas is suitable when dealing with medium-size files
df_pandas = pd.read_excel("/Workspace/Users/moraesit@outlook.com/tables_comments.xlsx",
                         engine='openpyxl', # When working with .xlsx files
                         sheet_name='deals',
                         header = 0)

df_deals = spark.createDataFrame(df_pandas)
df_deals.display()

In [0]:
from rich.console import Console

'''
Include the respective description inside each column's comments
'''
console = Console()
existing_columns = [col.name for col in spark.table("workspace.sc_silver.deals").schema]

for row in df_deals.collect():
    column = row['column_name']
    comment = row['description'] 

    if column in existing_columns:
        escaped_comment = comment.replace("'", "''")  # Escape single quotes to avoid SQL syntax errors
        spark.sql(f"""ALTER TABLE workspace.sc_silver.deals 
                      ALTER COLUMN {column} COMMENT '{escaped_comment}'""")
    else:
        console.print(f"Column [bold cyan]{column}[/bold cyan] doesn't exist in the deals table. Skipping...")

In [0]:
display(spark.sql("DESCRIBE workspace.sc_silver.deals"))